In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
country <- read_csv("../../data/country_data.csv")
population <- read_csv("../../data/population2022.csv")
covid <- read_csv("../../data/covid.csv")

Rows: 249 Columns: 11
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (7): name, alpha-2, alpha-3, iso_3166-2, region, sub-region, intermediat...
dbl (4): country-code, region-code, sub-region-code, intermediate-region-code

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 216 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): name, country-iso
dbl (1): population-2022

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 194 Columns: 3
── Column specification ────────────────

In [5]:
country |> head(3)

name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NA,142,34,NA
Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NA,150,154,NA
Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NA,150,39,NA


In [9]:
country <- country %>%
        select(name, `alpha-3`, region, `sub-region`, `intermediate-region`)

In [24]:
country %>% filter_all(any_vars(is.na(.))) %>% head(5)

name,alpha-3,region,sub-region,intermediate-region
<chr>,<chr>,<chr>,<chr>,<chr>
Afghanistan,AFG,Asia,Southern Asia,NA
Åland Islands,ALA,Europe,Northern Europe,NA
Albania,ALB,Europe,Southern Europe,NA
Algeria,DZA,Africa,Northern Africa,NA
American Samoa,ASM,Oceania,Polynesia,NA


In [27]:
country <- country %>% 
            mutate(`intermediate-region`= coalesce(`intermediate-region`, `sub-region`))

In [28]:
country %>% filter_all(any_vars(is.na(.))) %>% head(5)

name,alpha-3,region,sub-region,intermediate-region
<chr>,<chr>,<chr>,<chr>,<chr>
Antarctica,ATA,NA,NA,NA


In [30]:
country <- country %>% na.omit()

In [31]:
population %>% head(3)

name,country-iso,population-2022
<chr>,<chr>,<dbl>
Aruba,ABW,107697
Afghanistan,AFG,40683694
Angola,AGO,34907095


In [37]:
population %>% filter_all(any_vars(is.na(.))) %>% head(5)

name,country-iso,population-2022
<chr>,<chr>,<dbl>


In [6]:
covid.head(3)

,country-iso,confirmed,deaths
0,AFG,208324,7872
1,AGO,105184,1931
2,ALB,334135,3596


In [38]:
covid %>% filter_all(any_vars(is.na(.))) %>% head(5)

country-iso,confirmed,deaths
<chr>,<dbl>,<dbl>


In [46]:
df <- country %>% full_join(population, join_by(`alpha-3` == `country-iso`))

In [54]:
df <- df %>% mutate(name= coalesce(`name.x`, `name.y`))

In [65]:
df <- df %>% full_join(covid, join_by(`alpha-3` == `country-iso`))

In [67]:
df %>% head(5)

name.x,alpha-3,region,sub-region,intermediate-region,name.y,population-2022,name,confirmed,deaths
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
Afghanistan,AFG,Asia,Southern Asia,Southern Asia,Afghanistan,40683694,Afghanistan,208324,7872
Åland Islands,ALA,Europe,Northern Europe,Northern Europe,NA,NA,Åland Islands,NA,NA
Albania,ALB,Europe,Southern Europe,Southern Europe,Albania,2812325,Albania,334135,3596
Algeria,DZA,Africa,Northern Africa,Northern Africa,Algeria,45486083,Algeria,271364,6881
American Samoa,ASM,Oceania,Polynesia,Polynesia,American Samoa,54922,American Samoa,NA,NA


In [69]:
df <- df %>% select(-name.x, -name.y)

In [70]:
df <- na.omit(df)

In [79]:
df %>%
    group_by(region) %>%
    summarise(deaths=sum(deaths), confirmed=sum(confirmed), mr=100*deaths/confirmed) %>%
    arrange(desc(mr))

region,deaths,confirmed,mr
<chr>,<dbl>,<dbl>,<dbl>
Africa,257575,12493329,2.0617003
Americas,2913947,188273637,1.5477191
Europe,2018049,245894198,0.8206981
Asia,1589007,199796808,0.7953115
Oceania,22872,13690976,0.1670589


In [80]:
df %>%
    group_by(`sub-region`) %>%
    summarise(deaths=sum(deaths), confirmed=sum(confirmed), mr=100*deaths/confirmed) %>%
    arrange(desc(mr))

sub-region,deaths,confirmed,mr
<chr>,<dbl>,<dbl>,<dbl>
Northern Africa,88722,3780382,2.3469057
Latin America and the Caribbean,1755926,81454030,2.1557264
Sub-Saharan Africa,168853,8712947,1.9379551
Eastern Europe,861204,49311500,1.7464567
Southern Asia,772613,57989569,1.3323310
Central Asia,23818,1971023,1.2084080
Melanesia,1719,152072,1.1303856
Northern America,1158021,106819607,1.0840903
South-eastern Asia,365879,35593936,1.0279251


In [81]:
df %>%
    group_by(`intermediate-region`) %>%
    summarise(deaths=sum(deaths), confirmed=sum(confirmed), mr=100*deaths/confirmed) %>%
    arrange(desc(mr))

intermediate-region,deaths,confirmed,mr
<chr>,<dbl>,<dbl>,<dbl>
Central America,386115,11549141,3.3432357
Southern Africa,111621,4663341,2.3935843
Northern Africa,88722,3780382,2.3469057
South America,1345798,67523346,1.9930855
Eastern Europe,861204,49311500,1.7464567
Middle Africa,6620,445483,1.4860275
Eastern Africa,38442,2655549,1.4476103
Southern Asia,772613,57989569,1.3323310
Western Africa,12170,948574,1.2829784


In [104]:
r = df.groupby("region")[["confirmed", "population-2022"]].sum()
r["mr"] = 100*r["confirmed"]/r["population-2022"]
r.sort_values("mr", ascending=False)

,confirmed,population-2022,mr
region,,,
Europe,245894198.0,7.452442e+08,32.995117
Oceania,13690976.0,4.344141e+07,31.515955
Americas,188273637.0,1.031631e+09,18.250088
Asia,199796808.0,4.628024e+09,4.317108
Africa,12493329.0,1.403238e+09,0.890321
